In [1]:
using SymPy

In [41]:
M, m, g, k, kt, c, a, lG, lT, lf ,t, θ0, x0 = symbols("M, m, g, k, k_t, c, a, l_G, l_T, l_f , t, theta_0, x_0", real=true, positive=true)
x = SymFunction("x")(t)
θ = SymFunction("theta")(t)
f = SymFunction("f")(t)
;

In [20]:
ihat = [1,0,0]
jhat = [0,1,0]
khat = [0,0,1]
rG = lG*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_G*cos(theta(t))
 l_G*sin(theta(t))
                 0

In [21]:
rf = lf*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_f*cos(theta(t))
 l_f*sin(theta(t))
                 0

In [22]:
ra = a*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 a*cos(theta(t))
 a*sin(theta(t))
               0

In [23]:
rB = lT*(cos(θ)*ihat + sin(θ)*jhat)

3-element Vector{Sym}:
 l_T*cos(theta(t))
 l_T*sin(theta(t))
                 0

In [10]:
rm = rB - x*jhat

3-element Vector{Sym}:
        l_T*cos(theta(t))
 l_T*sin(theta(t)) - x(t)
                        0

In [17]:
dot(a,b) = sum(a .* b)
JG = 1//12*M*lT^2
T = 1//2*M*dot( diff.(rG,t), diff.(rG,t) ) + 1//2*JG*diff(θ,t)^2 + 1//2*m*dot( diff.(rm,t), diff.(rm,t)) |> simplify

                     2                        2     /                   2     
     2 /d           \         2 /d           \      |   2 /d           \      
M*l_G *|--(theta(t))|    M*l_T *|--(theta(t))|    m*|l_T *|--(theta(t))|  - 2*
       \dt          /           \dt          /      \     \dt          /      
---------------------- + ---------------------- + ----------------------------
          2                        24                                         

                                                    2\
                  d            d          /d       \ |
l_T*cos(theta(t))*--(theta(t))*--(x(t)) + |--(x(t))| |
                  dt           dt         \dt      / /
------------------------------------------------------
            2                                         

In [28]:
V = 1//2*kt*(θ - θ0)^2 + 1//2*k*(x - x0)^2 + M*g*dot(rG, jhat) + m*g*dot(rm, jhat)

                                                                        2     
                                                         k*(-x_0 + x(t))    k_
M*g*l_G*sin(theta(t)) + g*m*(l_T*sin(theta(t)) - x(t)) + ---------------- + --
                                                                2             

                       2
t*(-theta_0 + theta(t)) 
------------------------
          2             

In [29]:
L = T - V

                                              2                        2      
                              2 /d           \         2 /d           \       
                         M*l_G *|--(theta(t))|    M*l_T *|--(theta(t))|       
                                \dt          /           \dt          /       
-M*g*l_G*sin(theta(t)) + ---------------------- + ---------------------- - g*m
                                   2                        24                

                                                                              
                                                                              
                                             2                            2   
                              k*(-x_0 + x(t))    k_t*(-theta_0 + theta(t))    
*(l_T*sin(theta(t)) - x(t)) - ---------------- - -------------------------- + 
                                     2                       2                

  /                   2                           

In [37]:
rc = a*(cos(θ)*ihat + sin(θ)*jhat)
D = 1//2*c*dot(diff.(rc,t), jhat)^2

                                  2
 2      2           /d           \ 
a *c*cos (theta(t))*|--(theta(t))| 
                    \dt          / 
-----------------------------------
                 2                 

In [39]:
lag = [ diff(L, diff(q,t),t) - diff(L,q) + diff(D, diff(q,t)) |> simplify for q in [x,θ] ]

2-element Vector{Sym}:
                                                                                                                                               -g*m - k*(x_0 - x(t)) + m*(l_T*sin(theta(t))*Derivative(theta(t), t)^2 - l_T*cos(theta(t))*Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2)))
 M*g*l_G*cos(theta(t)) + M*l_G^2*Derivative(theta(t), (t, 2)) + M*l_T^2*Derivative(theta(t), (t, 2))/12 + a^2*c*cos(theta(t))^2*Derivative(theta(t), t) + g*l_T*m*cos(theta(t)) - k_t*theta_0 + k_t*theta(t) + l_T^2*m*Derivative(theta(t), (t, 2)) - l_T*m*cos(theta(t))*Derivative(x(t), (t, 2))

In [43]:
F = f*jhat
dot(F, diff.(rf, x) )

0

In [44]:
dot(F, diff.(rf, θ) )

l_f*f(t)*cos(theta(t))

In [45]:
Q = [ dot(F, diff.(rf, q) ) for q in [x,θ] ]

2-element Vector{Sym}:
                      0
 l_f*f(t)*cos(theta(t))

In [46]:
p1,p2 = symbols("p_1, p_2", real=true)

(p_1, p_2)

In [49]:
eqn1 = [ Eq(p1, diff(L, diff(x,t)) ),
    Eq(p2, diff(L, diff(θ,t)))]

2-element Vector{Sym}:
                                                                             Eq(p_1, m*(-2*l_T*cos(theta(t))*Derivative(theta(t), t) + 2*Derivative(x(t), t))/2)
 Eq(p_2, M*l_G^2*Derivative(theta(t), t) + M*l_T^2*Derivative(theta(t), t)/12 + m*(2*l_T^2*Derivative(theta(t), t) - 2*l_T*cos(theta(t))*Derivative(x(t), t))/2)

In [51]:
solp = solve(eqn1, [diff(x,t), diff(θ,t)])

Dict{Any, Any} with 2 entries:
  Derivative(x(t), t)     => (-12*M*l_G^2*p_1 - M*l_T^2*p_1 - 12*l_T^2*m*p_1 - …
  Derivative(theta(t), t) => (-12*l_T*p_1*cos(theta(t)) - 12*p_2)/(-12*M*l_G^2 …

In [54]:
H = T + V |> subs(solp) |> simplify

                                                                              
       /      2      2\                              2          2  2          
12*M*m*\12*l_G  + l_T /*(l_T*p_1*cos(theta(t)) + p_2)  + 144*l_T *m *(l_T*p_1*
------------------------------------------------------------------------------
                                                                              
                                                                              
                                                                              

                                                                              
                    2                                          /        2     
cos(theta(t)) + p_2)  - 24*l_T*m*(l_T*p_1*cos(theta(t)) + p_2)*\12*M*l_G *p_1 
------------------------------------------------------------------------------
                                                                              
                                                   

In [65]:
Qdamping = - [ diff(D, diff(q,t)) |> subs(solp) for q in [x,θ] ]

2-element Vector{Sym}:
                                                                                                                             0
 -a^2*c*(-12*l_T*p_1*cos(theta(t)) - 12*p_2)*cos(theta(t))^2/(-12*M*l_G^2 - M*l_T^2 + 12*l_T^2*m*cos(theta(t))^2 - 12*l_T^2*m)

In [69]:
pdot = [ -diff(H, q) for q in [x,θ] ] + Q  + Qdamping

2-element Vector{Sym}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [68]:
qdot = [ diff(H, p) for p in [p1,p2]]

2-element Vector{Sym}:
 (24*M*l_T*m*(12*l_G^2 + l_T^2)*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t)) + 288*l_T^3*m^2*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t)) - 24*l_T^2*m*(12*M*l_G^2*p_1 + M*l_T^2*p_1 + 12*l_T^2*m*p_1 + 12*l_T*m*p_2*cos(theta(t)))*cos(theta(t))^2 - 24*l_T*m*(l_T*p_1*cos(theta(t)) + p_2)*(12*M*l_G^2 + M*l_T^2 + 12*l_T^2*m)*cos(theta(t)) + (24*M*l_G^2 + 2*M*l_T^2 + 24*l_T^2*m)*(12*M*l_G^2*p_1 + M*l_T^2*p_1 + 12*l_T^2*m*p_1 + 12*l_T*m*p_2*cos(theta(t))))/(2*m*(12*M*l_G^2 + M*l_T^2 + 12*l_T^2*m*sin(theta(t))^2)^2)
                                                                                                                                                                                                                                                                                     (12*M*m*(12*l_G^2 + l_T^2)*(2*l_T*p_1*cos(theta(t)) + 2*p_2) - 288*l_T^2*m^2*(l_T*p_1*cos(theta(t)) + p_2)*cos(theta(t))^2 + 144*l_T^2*m^2*(2*l_T*p_1*cos(theta(t)) + 2*p_2))/(2*m*(12*M*l_G^

In [72]:
sympy.octave_code(pdot) |> println

% Not supported in Octave:
% f
% theta
% x
[g.*m - k.*(-2*x_0 + 2*x(t))/2; -a.^2.*c.*(-12*l_T.*p_1.*cos(theta(t)) - 12*p_2).*cos(theta(t)).^2./(-12*M.*l_G.^2 - M.*l_T.^2 + 12*l_T.^2.*m.*cos(theta(t)).^2 - 12*l_T.^2.*m) + 24*l_T.^2.*(12*M.*m.*(12*l_G.^2 + l_T.^2).*(l_T.*p_1.*cos(theta(t)) + p_2).^2 + 144*l_T.^2.*m.^2.*(l_T.*p_1.*cos(theta(t)) + p_2).^2 - 24*l_T.*m.*(l_T.*p_1.*cos(theta(t)) + p_2).*(12*M.*l_G.^2.*p_1 + M.*l_T.^2.*p_1 + 12*l_T.^2.*m.*p_1 + 12*l_T.*m.*p_2.*cos(theta(t))).*cos(theta(t)) + m.*(12*M.*l_G.^2 + M.*l_T.^2 + 12*l_T.^2.*m.*sin(theta(t)).^2).^2.*(2*M.*g.*l_G.*sin(theta(t)) + 2*g.*m.*(l_T.*sin(theta(t)) - x(t)) + k.*(x_0 - x(t)).^2 + k_t.*(theta_0 - theta(t)).^2) + (12*M.*l_G.^2.*p_1 + M.*l_T.^2.*p_1 + 12*l_T.^2.*m.*p_1 + 12*l_T.*m.*p_2.*cos(theta(t))).^2).*sin(theta(t)).*cos(theta(t))./(12*M.*l_G.^2 + M.*l_T.^2 + 12*l_T.^2.*m.*sin(theta(t)).^2).^3 + l_f.*f(t).*cos(theta(t)) - (-24*M.*l_T.*m.*p_1.*(12*l_G.^2 + l_T.^2).*(l_T.*p_1.*cos(theta(t)) + p_2).*sin(theta(t)) 